# Sky Model

In [1]:
demFile = r"Crater_Lake.tif"
skyConfig = r"..\SkyConfigFiles\Clear_300_NNW_45.txt"
ve = 4.0

In [2]:
import numpy as np
from PIL import Image
import matplotlib.pyplot as plt
from trf import utils, surface, shader, shadow

from ipywidgets import IntProgress, HTML, VBox
from IPython.display import display

cm=1/2.54

In [3]:
dem = np.asarray(Image.open(demFile))
#plt.imshow(dem, )

In [4]:
lights = utils.lightList(skyConfig)
snv = surface.normals_by_method(dem, 3.33, "N82")
dem = dem * ve

progressMsg = HTML()
progressBar = IntProgress(min=0, max=len(lights))
vbox=VBox(children=[progressBar, progressMsg])
display(vbox)

In [ ]:
accumulator = np.zeros(dem.shape)
for (i, (azimuth, elev, wt)) in enumerate(lights):
    progressBar.value = i
    progressMsg.value = f"Light {i} of {progressBar.max} : Az={azimuth} / El={elev} / Wt={wt}"
    shadowArray = shadow.shadowLine(dem, float(azimuth), float(elev), 3.33)
    shadeArray = shader.lambert(snv, utils.lightVector(float(azimuth), float(elev)))
    shadeArray[shadeArray<0] = 0
    shadeArray[shadowArray>0] = 0
    accumulator = accumulator + (float(wt) * shadeArray)
progressBar.bar_style='success'
hs = np.copy(accumulator / (accumulator.max() * 255))

In [ ]:
fig, axs = plt.subplots(1,1, figsize=(4*cm, 4*cm))
fig.set_dpi(600)
axs.yaxis.set_visible(False)
axs.xaxis.set_visible(False)

axs.imshow(hs, cmap='gray')